# Milestone 1: Data Preparation

---



Installing the necessary modules

In [ ]:
!pip install pytorch-lightning --quiet
!pip install wandb --quiet
!pip install kagglehub --quiet

Importing the necessary libs

In [ ]:
import pytorch_lightning as pl
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics.classification import Accuracy
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,random_split
from torchmetrics import Accuracy
import matplotlib.pyplot as plt
import os
from PIL import Image
import kagglehub
import cv2
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.loggers import WandbLogger
import wandb
from sklearn.utils.class_weight import compute_class_weight
import torch.optim as optim
import torch

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

Downloading the Alzheimer's dataset from Kaggle (https://www.kaggle.com/datasets/marcopinamonti/alzheimer-mri-4-classes-dataset?resource=download)

And plotting one image for sanity check

In [ ]:
# Download the dataset
alzheimer_dataset = kagglehub.dataset_download("marcopinamonti/alzheimer-mri-4-classes-dataset")

# Obtaining the root folder path
root_path = os.path.join(alzheimer_dataset, os.listdir(alzheimer_dataset)[0])

# Obtaining the class list
class_list = os.listdir(root_path)

# Plotting one image of the first class for checking
class_path = os.path.join(root_path, class_list[0])
image = os.path.join(class_path, "1 (2).jpg")
plt.imshow(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB))

Organizes and labels each image into its specific class and prepares data for the stratified K-Fold, which was chosen due to the unbalanced nature of the dataset

In [ ]:
# Creating the rows of data by matching images with their labels
rows = []
for label in sorted(class_list):
    label_dir = os.path.join(root_path, label)
    for img in os.listdir(label_dir):
        rows.append({"class_path": os.path.join(label_dir, img), "label": label})

# Creating the data frame with sorted values by label, and reseting the index
df = pd.DataFrame(rows).sort_values("label").reset_index(drop=True)

# Encode labels 0..3 and converts the names of the classes into numbers
le = LabelEncoder()
df["y"] = le.fit_transform(df["label"])

print("Class → index:", dict(zip(le.classes_, range(len(le.classes_)))))

Making Stratified K-Fold , which ensures that every fold has the same ratio of classes as the full dataset to make a fairer training and validation division.

In [ ]:
# We chose 3 folds
K = 3
X_idx = np.arange(len(df))
y = df["y"].to_numpy()

# Generating the 3 folds
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=70)

# Storing the validation and training indices for each fold
fold_indices = [(X_idx[tr], X_idx[va]) for tr, va in skf.split(X_idx, y)]
print(f"Prepared {len(fold_indices)} folds.")



Since our dataset is very unbalanced and we don't have enough images to test properly, we decided to divide the data into only training and validation sets. Below is the visualization of the 3 generated folds.



In [ ]:
def counts(arr):
    return dict(zip(le.classes_, np.bincount(arr, minlength=len(le.classes_))))

for i, (tr, va) in enumerate(fold_indices, start=1):
    print(f"\n=== Fold {i} ===")
    print("Train class counts:", counts(y[tr]))
    print("Val class counts:  ", counts(y[va]))


In [ ]:

# ---- Small dataset that reads from your df ----
class AlzheimerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):     # Every time the dataloader gets a new image, it will access the dataframe with iloc and then this image will be transformed
        row = self.df.iloc[idx]
        img = Image.open(row["class_path"]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = int(row["y"])
        return img, label     # The dataset will return the image and its label


class AlzheimerDataModule(pl.LightningDataModule):
    def __init__(
        self,
        df,
        fold_indices,
        fold_id=0,
        batch_size=32,
        use_weighted_sampler=True     # Due to the unbalanced dataset
    ):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.fold_indices = fold_indices
        self.fold_id = int(fold_id)
        self.batch_size = batch_size
        self.use_weighted_sampler = use_weighted_sampler

        # Makes sure all images are same size (128x128), transforms them to tensors and normalizes
        self.transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

        # Gets the number of classes
        self.num_classes = int(self.df["y"].nunique())

    def setup(self, stage=None):
        train_idx, val_idx = self.fold_indices[self.fold_id]

        # Base dataset from df
        base_ds = AlzheimerDataset(self.df, transform=self.transform)

        # Creates a smaller dataset from the base dataset using our obtained training and validation indexes
        self.train_data = Subset(base_ds, train_idx)
        self.val_data   = Subset(base_ds, val_idx)

        # Computes the weights for each class based on the number of images, so the classes with less images have higher weight values
        train_labels = self.df.iloc[train_idx]["y"].to_numpy()
        classes = np.arange(self.num_classes)
        cw = compute_class_weight(class_weight='balanced', classes=classes, y=train_labels)

        # Increases the punishment of the model in case it fails to correctly label the class with less images, since we have less data to train on it
        self.class_weights = torch.tensor(cw, dtype=torch.float32)
        print(f"Class Weights: {self.class_weights.tolist()}")

        # We are applying a weighted sampler to increase the loading of the class with less images to try and make the training more balanced
        if self.use_weighted_sampler:
            class_counts = np.bincount(train_labels, minlength=self.num_classes)
            per_class_w = 1.0 / class_counts
            sample_w = per_class_w[train_labels]
            self.train_sampler = WeightedRandomSampler(
                weights=torch.tensor(sample_w, dtype=torch.double),
                num_samples=len(sample_w),
                replacement=True
            )
        else:
            self.train_sampler = None


    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            shuffle=(self.train_sampler is None),     # In case there is no training sampler, we would shuffle the data
            sampler=self.train_sampler,               # However we are using a weighted sampler to try and increase the balance of the classes
            pin_memory=True                           # Optimizing the data transfer
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True
        )

In [ ]:
class AlzheimerLitModel(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=3e-4, class_weights=None):
        super().__init__()
        # save_hyperparameters saves the __init__ args automatically
        self.save_hyperparameters()

        self.input_shape = input_shape
        self.learning_rate = learning_rate

        # Feature extractor (convolutional layers)
        in_channels = input_shape[0]
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, 1, padding=1)
        self.pool1 = nn.MaxPool2d(2)
        self.pool2 = nn.MaxPool2d(2)

        # Compute flattened feature size after convs
        n_sizes = self._get_output_shape(input_shape)

        # Classifier
        self.fc1 = nn.Linear(n_sizes, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)

        # If class_weights provided, use them in the loss; convert to tensor later in setup
        self.register_buffer('class_weights_buf', torch.tensor(class_weights) if class_weights is not None else None)

        # Metrics
        self.train_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = Accuracy(task="multiclass", num_classes=num_classes)

    def _get_output_shape(self, shape):
        """Return flattened size after conv layers"""
        batch_size = 1
        device = next(self.parameters()).device if any(p.requires_grad for p in self.parameters()) else 'cpu'
        x = torch.zeros(batch_size, *shape).to(device)
        with torch.no_grad():
            x = F.relu(self.conv1(x))
            x = self.pool1(F.relu(self.conv2(x)))
            x = F.relu(self.conv3(x))
            x = self.pool2(F.relu(self.conv4(x)))
        n_size = x.view(batch_size, -1).size(1)
        return n_size

    def _feature_extractor(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        return x

    def forward(self, x):
        x = self._feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        logits = self.fc3(x)  # return raw logits
        return logits

    def _get_criterion(self):
        """Create loss function, using class weights if available."""
        if self.class_weights_buf is not None:
            return nn.CrossEntropyLoss(weight=self.class_weights_buf)
        return nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        criterion = self._get_criterion()
        loss = criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.train_acc(preds, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=False)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        criterion = self._get_criterion()
        loss = criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.val_acc(preds, y)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        criterion = self._get_criterion()
        loss = criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.test_acc(preds, y)
        self.log("test_loss", loss, on_epoch=True)
        self.log("test_acc", on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
# GEPETO DATA LOADING

fold_id = 0

data_module = AlzheimerDataModule(
    df=df,
    fold_indices=fold_indices,
    fold_id=fold_id,
    batch_size=32,
    use_weighted_sampler=True
)

# prepare_data is a no-op for this DM, but harmless if you want to keep the call
# data_module.prepare_data()
data_module.setup("fit")  # creates train/val Subsets and computes class weights

# ---- 1) Class weights and num_classes from the DataModule ----
# You no longer need to read labels from train_set; the DM already computed them.
class_weights = data_module.class_weights.cpu().numpy()  # ndarray[ num_classes ]
num_classes = data_module.num_classes
print("Class weights:", class_weights)

# ---- 2) Init model (RGB 128x128 as used in the DataModule transforms) ----
# Adjusted input_shape to (3, 128, 128) to match the transforms
model = AlzheimerLitModel(
    input_shape=(3, 128, 128),
    num_classes=num_classes,
    learning_rate=3e-4,
    class_weights=class_weights
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ---- 3) Loss and optimizer ----
# Use the same weights buffer the model already registered
criterion = nn.CrossEntropyLoss(weight=model.class_weights_buf)
optimizer = optim.Adam(model.parameters(), lr=model.learning_rate)

# ---- 4) Train loop (unchanged, just uses the DM loaders) ----
loss_values = []
eval_accuracy_values = []
num_epochs = 1 # Reduce epochs for faster testing

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in data_module.train_dataloader():
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == targets).sum().item()
        total += targets.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    loss_values.append(epoch_loss)
    eval_accuracy_values.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f} Accuracy: {epoch_acc:.4f}")